# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\L38 fiducial')
mouse_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\Mouse 1596')

registered_dir = Path(r'F:\Research\LINK\Mouse images\2019-09-28 Mouse 1596\Registered')

In [3]:
microscopy_path = Path(mouse_dir, 'SHG Mouse 1596 _ down8x.tif')
microscopy_tile_path = Path(mouse_dir, 'SHG Mouse 1596 settings.ome.tif')

us_path = Path(mouse_dir, 'L38 Mouse 1596 Z13422_Run-1.tif')
us_pl_path = Path(mouse_dir, 'L38 Mouse 1596 Z13422.pos')
us_param_path = Path(mouse_dir, 'L38 Mouse 1596 Z13422_Run-1_Settings.mat')

transform_path = Path(fiducial_dir, 'L38 Transform.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [15]:
spacing = [12, 12, 12]
dynamic_range = 80
gauge_height = 13422
us_image = coord.open_us(us_path, us_pl_path, us_param_path, spacing, dynamic_range, gauge_height)

In [6]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

Load the coordinate transform

In [7]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)

Apply the registration and save the results

In [8]:
us_reg_path = Path(registered_dir, 'Mouse 1596 - L38 to SHG.tif')
mpm_reg_path = Path(registered_dir, 'Mouse 1596 - SHG to L38.tif')

In [16]:
us_reg = tran.apply_transform_params(microscopy_image, us_image, 
                                     coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [17]:
mpm_reg = tran.apply_transform_params(us_image, microscopy_image, 
                                     -1*coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [18]:
sitk.WriteImage(us_reg, str(us_reg_path))

In [19]:
sitk.WriteImage(mpm_reg, str(mpm_reg_path))

In [21]:
us_rotated_path = Path(registered_dir, 'Mouse 1596 - L38 rotated.tif')
sitk.WriteImage(us_image, str(us_rotated_path))